In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, when
from datetime import datetime
import random

spark = SparkSession.builder.getOrCreate()

data = []
customers = ["Valerie Gray", "James Miller", "Ava Patel", "Olivia Clark"]
regions = ["North", "South", "East", "West"]
products = ["Monitor", "Smartphone", "Tablet"]
categories = ["Accessories", "Electronics"]

for i in range(150):  # 150 rows
    product = random.choice(products)
    row = {
        "OrderID": f"GEN{1000 + i}",
        "OrderDate": datetime.now().strftime("%-m/%-d/%Y"),
        "CustomerName": random.choice(customers) if random.random() > 0.1 else None,
        "Region": random.choice(regions),
        "Product": product,
        "Category": random.choice(categories),
        "Quantity": random.randint(1, 5),
        "UnitPrice": random.choice([689.7, 984.96, 499.99]),
    }
    row["Amount"] = round(row["Quantity"] * row["UnitPrice"], 2) if row["CustomerName"] else None
    data.append(row)

df = spark.createDataFrame(data)

# Save to raw container
output_path = f"abfss://raw-data@datapipelineprojectsa123.dfs.core.windows.net/generated_sales_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
df.write.mode("overwrite").option("header", True).csv(output_path)

print(f"✅ Synthetic sales file written to: {output_path}")
